In [ ]:
# !rm -rf ../data/df_raw/*
# !gsutil -m cp -r gs://np-training-tmp/stackoverflow/posts_processed/*.parquet ../data/raw/

In [18]:
from pyspark.sql import SparkSession
import re
import lxml.html
from pyspark.sql.types import StringType, ArrayType
import pyspark.sql.functions as F


In [2]:
spark = SparkSession\
.builder\
.appName('app')\
.config("spark.default.parallelism",200) \
.config("spark.sql.shuffle.partitions",200) \
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/17 10:15:33 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/17 10:15:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/17 10:15:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/17 10:15:34 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [5]:
posts_path = "gs://np-training-tmp/stackoverflow/output/posts__qn_with_top_answer/"
posts_link_path = "gs://np-training-tmp/stackoverflow/output/posts__links/"



In [6]:
df_posts = spark.read.parquet(posts_path)


In [9]:
df_posts.show(5)

+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+------------------+
|      Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|AnswerId|AcceptedAnswerBody|
+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+------------------+
|39457367|            null|How to configure ...|<p>I'm trying to ...|<java><eclipse><m...|     1048|          1|           1|    0|2016-09-12 19:24:...|    null|              null|
|52081324|            null|Apache 2.2 Cent O...|<p>I am trying to...|<apache><centos><...|       44|          0|           5|    0|2018-08-29 15:37:...|    null|              null|
|50452353|            null|Realm Java Equiva...|<p>In iOS with Re...|<java><android><r...|     

In [7]:
df_posts_link = spark.read.parquet(posts_link_path)

In [10]:
df_posts_link.show(5)

+--------+-------------+
|  PostId|RelatedPostId|
+--------+-------------+
|51490454|     30328586|
|22248511|     29987055|
|54632152|      3812055|
|46755225|      2472905|
|42799068|     22310912|
+--------+-------------+
only showing top 5 rows



In [34]:
regex = r"""
	<pre>.*?</pre>
	"""

def clean_text(snippet:str):
    
    if not snippet:
        return snippet
    snippet = re.sub(pattern=regex, repl = '[CODE]', string = snippet,  flags = re.IGNORECASE | re.DOTALL | re.MULTILINE | re.VERBOSE )
    
    snippet = str(lxml.html.fromstring(snippet).text_content())
    
    return snippet

def parse_tags(content:str):
    return re.findall(r'<(.+?)>',content)


udf_clean_text = F.udf(lambda x:clean_text(x),StringType() )
udf_parse_tags = F.udf(lambda x:parse_tags(x),ArrayType (StringType() ))

In [35]:
clean_text(f"""

<p>I was asked to create a singleton that will..</p>.

<pre><code>KDF </code></pre>

<p> test </p>

""")

'I was asked to create a singleton that will...\n\n[CODE]\n\n test \n\n'

In [37]:
clean_text('')

''

In [38]:
clean_text(None)

In [51]:
df_posts_processed = df_posts \
                        .withColumn("QuestionBody",udf_clean_text(F.col("QuestionBody"))) \
                        .withColumn("AcceptedAnswerBody",udf_clean_text(F.col("AcceptedAnswerBody"))) \
                        .withColumn("Tags",udf_parse_tags(F.col("Tags"))) \
                        .cache()


In [52]:
df_posts_processed.where(F.col("AcceptedAnswerBody").isNotNull()).show(5)

+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|      Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|AnswerId|  AcceptedAnswerBody|
+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|70623033|        70626284|bash: cat + grep ...|Using Linux bash ...|   [bash, grep, cat]|       58|          3|           0|    1|2022-01-07 14:52:...|70626284|Here is an awk so...|
|46776107|        46776332|Ajax getting 400 ...|I am trying to PO...|[javascript, html...|     2637|          4|           7|    0|2017-10-16 17:43:...|46776332|Since you are try...|
|68080665|        68081670|Beautiful Soup fi...|So, I try to scra...|[python, html, w

In [ ]:
pdf_top_tags = df_posts_processed\
.where(F.col("AcceptedAnswerBody").isNotNull()) \
.select(F.explode("Tags").alias('tag')) \
.groupby(['tag']) \
.count() \
.toPandas() \
.sort_values(['count'], ascending=False)

In [ ]:
pdf_top_tags

In [ ]:
df_posts_link \
.join(df_posts_processed, on =[''])

In [ ]:
df2 = df.select(df.name,explode(df.knownLanguages))


In [27]:
df = df_raw

NameError: name 'df_raw' is not defined

In [ ]:
df['QuestionBody'] =  df['QuestionBody'].apply(strip_html)

In [ ]:
df['AnswerBody'] =  df['AnswerBody'].apply(strip_html)

In [ ]:
df['Tags'] = df['Tags'].apply(parse_tags)

In [ ]:
!rm -rf ../data/df_processed/

In [ ]:
df.to_parquet("../data/df_processed/")